# Intro to Deep Learning. Chapter 2: Learning


## <span id="secContent">Content</span>

1. [Summary of Chap1's code]()
2. [The Training: First Learning Step](#secTheTraining)




In [59]:
##Summary of Chap1
#
#
from math import floor
import numpy as np

def synapticPotentialWeighted(X,W,b=0):
    return np.dot(X,W[:len(X)])+b
    
def synapticPotentialAverage(X,W=None):
    return sum(X)/len(X)
    
def synapticPotentialMedian(X,W=None):
    """
    To make sense, 
    WT := \sum |W_i| := total # of X values (not all distinct) seen.
    Assing each X_i to its weight W_i -> (X_i,W_i)
    sort by X_i
    lx2=floor(WT/2)
    if exists j ; |W_j| = lx2 :
        md = ( W_j*X_j + W_(j+1) *X_(j+1) ) / ( W_j + W_(j+1)) #weighted average of both points
    if WT%2==0: 
    else:
        md
        
    Example: w1=7 w2=5 w3=2 => WT=14 => lx2=7 => md = (w1*x1+w2*x2 ) / (w1+w2) = (7x1+5x2)/12
                  
    if lx%2 == 0 ):
        md=(Z[lx2-1] + Z[lx2] )*0.5
    else:
        md=Z[lx2]*1.0 #so that it always returns a float
    """
    
    # W=[1,1,...,1] as many 1's as length of X
    if W == None: 
        W=np.ones(len(X)) 

    #Given array [x1,x2,...xn] -> get the sum_i xi
    def foldr(_X,_accu=0):          
        if len(_X) == 0: return _accu
        if len(_X) == 1: return _accu+_X[0]
        return (_accu + _X[-1] + foldr(_X[:-1]) )

    WT = foldr( list( map(abs,W) ) )
    
    def pfilter(par,x):
        for a,b in par:
            if abs(b) == x: return par.index((a,b))
        return None

    Z=sorted(list(zip(X,W)))
    lx2=floor(WT/2)
    #on Z=sorted(X,W): Python will mutate arguments **if it can**
               
    #if WT%2 == 0:
    #    j = pfilter(Z,lx2)
    #    try: return (Z[j][1]*Z[j][0]+Z[j+1][1]*Z[j+1][0]) 
    #    except: print('ERROR j= ',j," Z: ",Z, " WT: ",WT)
    #else:
    j=0 ; zw=abs(Z[j][1])
    while zw < lx2:
        j+=1
        zw+=abs(Z[j][1]) 
    if j == len(Z)-1: j-=1
    
    try: return (Z[j+1][1]*Z[j+1][0]+Z[j][1]*Z[j][0])/(abs(Z[j+1][1])+abs(Z[j][1])) # {wj*xj + w(j+1)*x(j+1)}/(wj+wj+1) 
    except: print('ERROR j=',j," Z: ",Z, " WT: ",WT)

def synapticPotentialMode(X,W):
    """
        To make sense, the weights W should be non-negative.
    Choose randomly one among all possible maxima
    
    When W[1,1] Provides a soft boundary between both phases (classes) with difussion across the boundary
    """
    wmax = max(list(map(abs,W)) )
    imax = [ i for i,j in enumerate(list(map(abs,W)) ) if j==wmax ]
    #print("max weights:",imax)
    i = np.random.choice(imax,1)
    return X[i]*W[i]/wmax
    
##
def nonLinearBipolarStep(x,string=None):
    if not string: return (-1 if x<0 else 1 )
    else: return ('-' if x<0 else '1')

def nonLinearHeaviside(x,string=None):
    if not string: return (0 if x<0 else 1 )
    else: return ('o' if x<0 else '1')

def nonLinearRelu(x,string=None):
    r = max(0,x)
    if not string: return r
    else: return str(r)

def identity(x,string=None):
    if not string: return x
    else: return str(x)

## General perceptron: called ng_nGenPerceptron in previous Chapter
def perceptron(X,W,B=[0],
                   synapticPotential=synapticPotentialWeighted,
                   #activation=nonLinearBipolarStep,
                   activation=nonLinearHeaviside,
                   #activation=(lambda z,s: nonLinearBipolarStep(nonLinearRelu(z,None),s)),
                   string=True,debug=False,showHidden=False):
    if B == None: B=[0]
    if debug: print("ng_nGenPer: W:",W," X:",X," B:",B)    
    return activation(synapticPotential(X,W,b=B[0]),string=string)

## (Feed-forward neural network for XOR function using Bipolar step as activation function
##   and weighted sum as aggregator)
def nnXOR(X,W,showHidden=None,debug=False):
    assert len(W)==6 
    strings=[False,False,False,False,False]
    h=0
    if showHidden : 
        h=(showHidden-1)%2 # h1-2
        assert h>=0
        strings[h]=True 
        
    hl=[]
    hl.append(perceptron(X=X,W=W[0:2],B=[0],
                        synapticPotential=synapticPotentialWeighted,
                        activation=nonLinearBipolarStep,
                        string=strings[0],debug=debug
                        ) 
             )
    hl.append(perceptron(X=X,W=W[2:4],B=[0],
                        synapticPotential=synapticPotentialWeighted,
                        activation=nonLinearBipolarStep,
                        string=strings[1],debug=debug
                        ) 
             )

    if showHidden: return hl[h]
    
    return nPerceptron(X=[hl[0],hl[1]],W=W[4:6],string=True)

## AND function
def perceptronAND(X):
    return perceptron(X=X,W=[1,1],B=[2],
                             synapticPotential=synapticPotentialWeighted,
                             activation=nonLinearHeaviside,
                             string=False,debug=False,showHidden=False)

## Neural network for a convex hull pattern
def nnConvexHull(X,W,B=[0,0,0,0],
                 synapticPotential=synapticPotentialWeighted,
                 activation=nonLinearHeaviside,
                 string=None,debug=False,showHidden=False):
    """
    Classifies the 2d plane in two distinct regions with an arbitrary polygonal perimetre
    """
    hl=[0,0,0,0]
    for i in range(len(B)):
        hl[i] = perceptron(X,W=W[2*i:2*i+2], B=B[i:i+1],
                            synapticPotential=synapticPotential,
                            activation=activation,
                            string=string,debug=debug,showHidden=showHidden)
    if debug: 
        print(hl[0],hl[1],hl[2],hl[3])
        
    return perceptronAND( [perceptronAND( [hl[0],hl[1]] ), perceptronAND( [hl[2],hl[3]] ) ] )


## Helper function. called ng_scan2InputNN in previous chapter
def scan2InputNN(nn,weights,Biases=None,xysup=11,showHidden=None,debug=False): 
    """
    ng_scan2InputNN(nn,weights,xysup=11,showHidden=None,debug=False)
    
    Same as original scan2InputNN but takes an additional parameter b, the bias of the perceptrons. 
    The bias b is an array of numbers for each of the different perceptrons the NN is  composed of.
    
    Pass a 'neural network with 2 inputs and its weights, then scan a given range of values for those inputs and see 
     where the NN classifies those points into ('-' / '+')
     
    NN needs to have defined inputs and weights as simple array. Also optional variables showHidden, 
    for showing values of hidden neurons, or debug for additional info
    
    xysup=11 #supremum of x,y ranges, with -xysup(1-[1/2]) <= x,y <= xysup(1-[1/2]). 
    E.g., xysup=11, xy \in [-5,5]
    
    It calls the NN as: nn([x1,x2],weights,showHidden=showHidden,debug=debug)
    """
    X1 = np.arange(0,xysup)
    X2 = np.arange(0,xysup)
    X1 -= floor(xysup/2)
    X2 -= floor(xysup/2)
    X1

    if showHidden: print("Showing hidden neuron ",showHidden)
        
    print("      ",end="") #formatting
    for x1 in X1:          #Print first line w/ x-coord 
        print(x1,end="  ")
    print("\n\t",end="") 
    for x2 in reversed(X2):                               #Print first y coor. (+ values top/first=>reverse array)
        print(x2,end=" ") if x2<0 else print(x2,end="  ")
        for x1 in X1:                                     #...then go on w/ values along x-axis
            #print(nn1HiddenLayer3([x1,x2],W),end="  " )
            #if debug: help(nn)
            print(nn([x1,x2],weights,Biases,showHidden=showHidden,debug=debug),end="  " )
        print("\n\t",end="")


# <span id="secTheTraining">The Training: First Learning Step</span>[^](#secContent)

We would like to define a *training* procedure that would allow any given network to *learn* the values of the weights and biases by simply giving the network a bunch of example inputs and their expected output. Ideally, at the end of the this *training phase* the resulting weights would make the our neural network nail all answers for all inputs.

But, **how can the neural network *learn*?** Well, this question can now be recast as **what training steps can we device that, given the examples, yields the right set of weights?**

In [52]:
def trainNN(nn,trainingSet,W,nbInputNeurons=2):
    '''
    How nn can learn?
    X'1, X'2,...X'm (vectors n-dim) input examples + Y'1,...,Y'm (K-dim) outputs
    Idea: Let's define a "learning"/"training" method as follows:
    0.- Assume NN topology fixed.
    1.- Choose weights at random, small.
    2.- Get Yi i=1,..,m output vectors for each input X'i
    3.- Decide how good those m outputs are => Criterion ??
    4.- If good enough then FINISHED; otherwise: Change new weights (hopefully so as to improve 3)
    5.- Go to step 2.

    Example of possible Criterion: \Sum_i (Yi - Y'i)^2 := Euclidean distance

    How change weights (step 4)?Newton (Gradient) + Back Propagation

    wij(t+1) = wij(t) - \eta (yi-y'i)*xj
    
    X1o
       \    
        \w1           __
         \__________ |  _____o y
         /          _|
        /w2  
       /    
    X2o
      
    '''


## Example: Learning the AND function

Let's try a quick and dirty procedure. In order to keep things simple, we will set to try learning the AND operator.


In [66]:
## Testing naive training method
## Try learning AND function => simple perceptron
tset=[([0,0],0),([1,0],0),([0,1],0),([1,1],1)]

weights=np.array([0.1,0.2])
bias=np.array([0.3])
#before learning AND function from examples
scan2InputNN(perceptron,weights=weights,Biases=bias,xysup=3)
print('No info on AND function. NN yields 1 for all four inputs',end="\n\n")

#Start learning
print("Start learning/training step:",'weights / bias / Total Error')
eta=0.01
debug=False
nIterations=20
for i in range(nIterations): #we repeat. Goal: hopefully after studying example many times it stops making mistakes E=0-
    E=0
    for x,yt in tset:
        if debug: print(x,yt,weights,bias," : ",end="")
        y=perceptron(X=x,W=weights,B=bias,activation=nonLinearHeaviside,string=False)
        E += (y-yt)**2
        if debug: print(y,y-yt,E)
        xa = np.array(x)
        weights -= eta*(y-yt)*xa
        bias[0] -= eta*(y-yt)

    print("End of iteration i="+str(i),weights,bias," : E=",E)

#After learning
print('Finished',nIterations,'iterations of learning/training')
scan2InputNN(perceptron,weights=weights,Biases=bias,xysup=3)

      -1  0  1  
	1  1  1  1  
	0  1  1  1  
	-1 o  1  1  
	No info on AND function. NN yields 1 for all four inputs

Start learning/training step: weights / bias / Total Error
End of iteration i=0 [ 0.09  0.19] [ 0.27]  : E= 3
End of iteration i=1 [ 0.08  0.18] [ 0.24]  : E= 3
End of iteration i=2 [ 0.07  0.17] [ 0.21]  : E= 3
End of iteration i=3 [ 0.06  0.16] [ 0.18]  : E= 3
End of iteration i=4 [ 0.05  0.15] [ 0.15]  : E= 3
End of iteration i=5 [ 0.04  0.14] [ 0.12]  : E= 3
End of iteration i=6 [ 0.03  0.13] [ 0.09]  : E= 3
End of iteration i=7 [ 0.02  0.12] [ 0.06]  : E= 3
End of iteration i=8 [ 0.01  0.11] [ 0.03]  : E= 3
End of iteration i=9 [  1.04083409e-17   1.00000000e-01] [ -1.42247325e-16]  : E= 3
End of iteration i=10 [  1.04083409e-17   9.00000000e-02] [-0.01]  : E= 1
End of iteration i=11 [  1.04083409e-17   8.00000000e-02] [-0.02]  : E= 1
End of iteration i=12 [  1.04083409e-17   7.00000000e-02] [-0.03]  : E= 1
End of iteration i=13 [  1.04083409e-17   6.00000000e-02] 

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:107: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


<pre>
    X1o_____________| _o
       \     /
        \   /
         \ /
         /\
        /  \
       /    \
      /      \
    X2o-------------| -o

</pre>